### Scifi TV Shows

In [ ]:
!unzip '/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip'

Archive:  /content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip
   creating: OriginalStoriesSeparated/
  inflating: OriginalStoriesSeparated/Babylon5.txt  
  inflating: OriginalStoriesSeparated/DoctorWhoSpinoffs.txt  
  inflating: OriginalStoriesSeparated/DoctorWho.txt  
  inflating: OriginalStoriesSeparated/Fringe.txt  
  inflating: OriginalStoriesSeparated/Farscape.txt  
  inflating: OriginalStoriesSeparated/Futurama.txt  
  inflating: OriginalStoriesSeparated/Stargate.txt  
  inflating: OriginalStoriesSeparated/StarTrek.txt  
  inflating: OriginalStoriesSeparated/StarWarsBooks.txt  
  inflating: OriginalStoriesSeparated/StarWarsRebels.txt  
  inflating: OriginalStoriesSeparated/XFiles.txt  
  inflating: scifi-test.txt          
  inflating: scifi-train.txt         
  inflating: scifi-val.txt           


### CMU Summaries

In [1]:
import tarfile
my_tar = tarfile.open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/MovieSummaries.tar.gz')
my_tar.extractall('./CMU_Summaries')
my_tar.close()

In [1]:
with open('/content/CMU_Summaries/MovieSummaries/README.txt','r') as f:
  readme_data = f.read()
# print(readme_data)

In [2]:
import pandas as pd
df_movie_meta = pd.read_table('/content/CMU_Summaries/MovieSummaries/movie.metadata.tsv', header=None, names = ['Wikipedia movie ID','Freebase movie ID','Movie name','Movie release date','Movie box office revenue','Movie runtime','Movie language','Movie countries','Movie genres'])

In [ ]:
df_movie_meta

In [3]:
df_charac_meta = pd.read_table('/content/CMU_Summaries/MovieSummaries/character.metadata.tsv',header=None, names = ['Wikipedia movie ID', 'Freebase movie ID','Movie release date','Character name','Actor date of birth','Actor gender','Actor height','Actor ethnicity','Actor name','Actor age at movie release','Freebase character/actor map ID','Freebase character ID','Freebase actor ID'])

In [ ]:
df_charac_meta

In [4]:
with open('/content/CMU_Summaries/MovieSummaries/plot_summaries.txt','r') as f:
  summaries = f.read()

In [5]:
summaries = summaries.split('\n')

In [ ]:
summaries

In [6]:
df_summaries = pd.DataFrame(columns=['Wikipedia movie ID', 'Summary'])

In [7]:
df_summaries['Summary'] = summaries

In [8]:
def sep_id(x):
  return x.split('\t')[0]
def sep_summary(x):
  try:
    return x.split('\t')[1]
  except:
    return None

In [9]:
df_summaries['Wikipedia movie ID']  = df_summaries['Summary'].apply(sep_id)
df_summaries['Summary'] = df_summaries['Summary'].apply(sep_summary)

In [10]:
df_summaries.dropna(inplace=True)

In [11]:
df_summaries.reset_index(inplace=True, drop=True)

In [ ]:
#Combining movie summaries with other information - movie and character metadata 

In [12]:
df_summaries = df_summaries.assign(Genre="", Movie_Name="",Characters="")

In [ ]:
df_summaries

In [13]:
labels = []
for idx, row in df_summaries.iterrows():
  # for idx2, row2 in df_movie_meta.iterrows():
    #Get index and then assign values, avoid the for loop and if condition (do the same for characters)
    # if row2['Wikipedia movie ID'] == row['Wikipedia movie ID']:
      # df_summaries['Movie_Name'].iloc[idx] = row2['Movie name']
      # df_summaries['Genre'].iloc[idx] = row2['Movie genres']
      # break
  try: 
    index = df_movie_meta['Wikipedia movie ID'].values.tolist().index(int(row['Wikipedia movie ID']))
    df_summaries['Movie_Name'].iloc[idx] = df_movie_meta['Movie name'].iloc[index]
    df_summaries['Genre'].iloc[idx] = df_movie_meta['Movie genres'].iloc[index]
  except:
    labels.append(idx)

In [ ]:
labels

In [ ]:
len(labels)

In [ ]:
df_summaries

In [14]:
df_summaries.drop(index=labels, axis=0, inplace=True)
df_summaries.reset_index(drop=True, inplace=True)

In [ ]:
# Extract Genre 

In [15]:
labels=[]
for idx, row in df_summaries.iterrows():
  index=-1
  check = True
  # for idx2, row2 in df_charac_meta.iterrows():
  #   if row2['Wikipedia movie ID'] == row['Wikipedia movie ID']:
  #     df_summaries['Characters'].iloc[idx] = df_summaries['Characters'].iloc[idx]+row2['Character name']
  print()
  print(row['Movie_Name'], end=' ')
  try:  
    while True:
      index = df_charac_meta['Wikipedia movie ID'].values.tolist()[index+1:].index(int(row['Wikipedia movie ID']))+index+1
      df_summaries['Characters'].iloc[idx] = df_summaries['Characters'].iloc[idx] + df_charac_meta['Character name'].iloc[index]
      print(df_charac_meta['Character name'].iloc[index], end=' ')
      check = False
  except:
    if check:
      labels.append(idx)

In [16]:
df_summaries

,Wikipedia movie ID,Summary,Genre,Movie_Name,Characters
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...","{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...",Taxi Blues,
1,31186339,The nation of Panem consists of a wealthy Capi...,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":...",The Hunger Games,FoxfaceKatniss EverdeenPeeta MellarkEffie Trin...
2,20663735,Poovalli Induchoodan is sentenced for six yea...,"{""/m/04t36"": ""Musical"", ""/m/02kdv5l"": ""Action""...",Narasimham,M.K. Menon
3,2231378,"The Lemon Drop Kid , a New York City swindler,...","{""/m/06qm3"": ""Screwball comedy"", ""/m/01z4y"": ""...",The Lemon Drop Kid,
4,595909,Seventh-day Adventist Church pastor Michael Ch...,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",A Cry in the Dark,
...,...,...,...,...,...
42202,34808485,"The story is about Reema , a young Muslim scho...","{""/m/0bj8m2"": ""Children's""}",Oomakkuyil Padumbol,
42203,1096473,"In 1928 Hollywood, director Leo Andreyev look...","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",The Last Command,
42204,35102018,American Luthier focuses on Randy Parsons’ tra...,"{""/m/02hmvc"": ""Short Film"", ""/m/04rlf"": ""Music...",Randy Parsons: American Luthier,
42205,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se...","{""/m/07s9rl0"": ""Drama""}",Kabuliwala,LakshmiAbdul Rehman KhanMiniRama


In [17]:
len(labels)

28289

In [ ]:
# Take characters from BookNLP instead of the corpus